In [ ]:
ATLAS_API_KEY = "{PASTE-API-KEY-HERE}"

In [2]:
from ripe.atlas.cousteau import (
  Ping,
  Traceroute,
  Dns,
  AtlasSource,
  AtlasCreateRequest
)
from ripe.atlas.cousteau import AtlasStopRequest

In [3]:
import csv
import os
import sys
import requests
import json
import time

In [4]:
import ipaddress
from datetime import timedelta, date, datetime

In [5]:
%matplotlib inline
from datetime import datetime
import pandas as pd
import sqlite3
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from ripe.atlas.cousteau import Measurement
import seaborn as sns
pd.options.display.max_colwidth = 100

In [21]:
db = sqlite3.connect("data/ripe-dns-all-probes-with-abuf.db")
df = pd.read_sql_query("SELECT * from data", db)
db.close()

In [17]:

resolvers_dict = {'185.228.168.168':'CleanBrowsing', '1.1.1.1':'Cloudflare', '8.8.8.8':'Google',
                  '208.67.222.123':'OpenDNS', '185.121.177.177':'OpenNIC', '9.9.9.9':'Quad9', '64.6.64.6':'VeriSign',
                  '77.88.8.8':'Yandex', '156.154.70.1':'Neustar UltraRecursive', '45.90.28.0':'NextDNS',
                  '2a0d:2a00:1::1':'CleanBrowsing', '2606:4700:4700::1111':'Cloudflare',
                  '2001:4860:4860::8888':'Google', '2620:0:ccc::2':'OpenDNS',
                  '2a05:dfc7:5::5353':'OpenNIC', '2620:fe::9':'Quad9', '2620:74:1b::1:1':'VeriSign',
                  '2a02:6b8::feed:ff':'Yandex', '2610:a1:1018::1':'Neustar UltraRecursive',
                     '2a07:a8c0::': 'NextDNS'}

def is_global_ip(ipaddr):
    try:
        return ipaddress.ip_address(ipaddr).is_global
    except (ValueError):
        return False
    
def centralized_resolvers(ipaddr):
    if ipaddr in list(resolvers_dict.keys()):
        return True
    else:
        return False
    
def local_centralized_resolvers(ipaddr, is_local):
    if is_local == 'True':
        if ipaddr in list(resolvers_dict.keys()):
            return 'local and centralized'
        else:
            return 'local and not centralized'
    else:
        return 'not local'    


In [22]:
rslv_list = df.groupby('resolver_address').agg({'prb_id':pd.Series.nunique}).index.tolist()
rslv_list.pop(0)
ctr = 0
rslv_res = []
for rslv in rslv_list:
    print(str(ctr), end=' ')
    rslv_res.append((rslv, is_global_ip(rslv), centralized_resolvers(rslv)))
    ctr = ctr + 1
df_metadata = pd.DataFrame(rslv_res, columns = ['resolver_address', 'rslv_ip_is_global', 'local_and_centralized_rslv'])
df['dtime'] = pd.to_datetime(df['timestamp'], unit='s')
df = pd.merge(df, df_metadata, how='left', on=['resolver_address'])
df.rename(columns={'local_and_centralized_rslv': 'centralized_rslv'}, inplace=True)

df_provison_full = df[(df.local_resolver == 'True') & (df.rslv_ip_is_global == True) & (df.centralized_rslv == False)].groupby(['resolver_address', 'address_family']).agg({'prb_id':lambda x: set(x)}).reset_index()


for i in range(0, len(df_provison_full)):
    resolver = df_provison_full.loc[i].resolver_address
    addr_format = '6' if resolver.find(':') > 0 else '4'
    if df_provison_full.loc[i].address_family != addr_format:
        print(df_provison_full.loc[i].address_family)
        #print(addr_format)
        print(resolver)
        print(df_provison_full.loc[i])

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

 2102 2103 2104 2105 2106 2107 2108 2109 2110 2111 2112 2113 2114 2115 2116 2117 2118 2119 2120 2121 2122 2123 2124 2125 2126 2127 2128 2129 2130 2131 2132 2133 2134 2135 2136 2137 2138 2139 2140 2141 2142 2143 2144 2145 2146 2147 2148 2149 2150 2151 2152 2153 2154 2155 2156 2157 2158 2159 2160 2161 2162 2163 2164 2165 2166 2167 2168 2169 2170 2171 2172 2173 2174 2175 2176 2177 2178 2179 2180 2181 2182 2183 2184 2185 2186 2187 2188 2189 2190 2191 2192 2193 2194 2195 2196 2197 2198 2199 2200 2201 2202 2203 2204 2205 2206 2207 2208 2209 2210 2211 2212 2213 2214 2215 2216 2217 2218 2219 2220 2221 2222 2223 2224 2225 2226 2227 2228 2229 2230 2231 2232 2233 2234 2235 2236 2237 2238 2239 2240 2241 2242 2243 2244 2245 2246 2247 2248 2249 2250 2251 2252 2253 2254 2255 2256 2257 2258 2259 2260 2261 2262 2263 2264 2265 2266 2267 2268 2269 2270 2271 2272 2273 2274 2275 2276 2277 2278 2279 2280 2281 2282 2283 2284 2285 2286 2287 2288 2289 2290 2291 2292 2293 2294 6
195.130.131.1
resolver_address  

In [18]:
rslv_list = df.groupby('resolver_address').agg({'prb_id':pd.Series.nunique}).index.tolist()
rslv_list.pop(0)
ctr = 0
rslv_res = []
for rslv in rslv_list:
    print(str(ctr), end=' ')
    rslv_res.append((rslv, is_global_ip(rslv), centralized_resolvers(rslv)))
    ctr = ctr + 1
df_metadata = pd.DataFrame(rslv_res, columns = ['resolver_address', 'rslv_ip_is_global', 'local_and_centralized_rslv'])
df['dtime'] = pd.to_datetime(df['timestamp'], unit='s')
df = pd.merge(df, df_metadata, how='left', on=['resolver_address'])
df.rename(columns={'local_and_centralized_rslv': 'centralized_rslv'}, inplace=True)

df_provison_full = df[(df.local_resolver == 'True') & (df.rslv_ip_is_global == True) & (df.centralized_rslv == False)].groupby(['resolver_address']).agg({'prb_id':lambda x: set(x)}).reset_index()



0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

 2032 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046 2047 2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060 2061 2062 2063 2064 2065 2066 2067 2068 2069 2070 2071 2072 2073 2074 2075 2076 2077 2078 2079 2080 2081 2082 2083 2084 2085 2086 2087 2088 2089 2090 2091 2092 2093 2094 2095 2096 2097 2098 2099 2100 2101 2102 2103 2104 2105 2106 2107 2108 2109 2110 2111 2112 2113 2114 2115 2116 2117 2118 2119 2120 2121 2122 2123 2124 2125 2126 2127 2128 2129 2130 2131 2132 2133 2134 2135 2136 2137 2138 2139 2140 2141 2142 2143 2144 2145 2146 2147 2148 2149 2150 2151 2152 2153 2154 2155 2156 2157 2158 2159 2160 2161 2162 2163 2164 2165 2166 2167 2168 2169 2170 2171 2172 2173 2174 2175 2176 2177 2178 2179 2180 2181 2182 2183 2184 2185 2186 2187 2188 2189 2190 2191 2192 2193 2194 2195 2196 2197 2198 2199 2200 2201 2202 2203 2204 2205 2206 2207 2208 2209 2210 2211 2212 2213 2214 2215 2216 2217 2218 2219 2220 2221 2222 2223 2224 2225 2226 2227 2228 2229 2230 2231

In [19]:
df_provison_full

,resolver_address,prb_id
0,0:0:53f9:459f::1,{54990}
1,1.0.0.1,"{52224, 54528, 17154, 24066, 32770, 32005, 11526, 12929, 54154, 23308, 52621, 51727, 55567, 5531..."
2,1.0.0.3,"{33090, 53335}"
3,1.1.0.1,{21228}
4,1.1.1.2,"{16017, 55313, 17318}"
...,...,...
1082,94.254.0.147,{54426}
1083,95.131.96.254,{26911}
1084,95.56.254.102,{34654}
1085,96.76.225.35,{11513}


In [26]:
for i in range(0, len(df_provison_full)):
    probe_ids = list(df_provison_full.loc[i].prb_id)
    source = AtlasSource(
        type = "probes",
        value = ','.join(str(id) for id in probe_ids),
        requested = len(probe_ids)
    )
    resolver = df_provison_full.loc[i].resolver_address
    msm_list = []
    addr_format = '6' if resolver.find(':') > 0 else '4'
    #addr_format = df_provison_full.loc[i].address_family
    if addr_format == '6':
        msm_list.append(Traceroute(af=6, target=resolver, description="ICMP Traceroute IPv6 to " + resolver + ".", resolve_on_probe=True, protocol="ICMP", packets=1, size=0, first_hop=1, max_hops=64, paris=16, port=80))
    elif addr_format == '4':
        msm_list.append(Traceroute(af=4, target=resolver, description="ICMP Traceroute IPv4 to " + resolver + ".", resolve_on_probe=True, protocol="ICMP", packets=1, size=0, first_hop=1, max_hops=64, paris=16, port=80))
    
       
    atlas_request = AtlasCreateRequest(
        start_time = datetime.utcnow() + timedelta(minutes=2),
        key = ATLAS_API_KEY,
        measurements = msm_list,
        sources = [source],
        is_oneoff = True)
    
    (is_success, response) = atlas_request.create()
        
        
    if is_success:
        print("Success!")
        print(len(response['measurements']))
        #If successful write msm_id to csv
        with open('metadata/msm_id_list_traces_to_local_resolvers_with_af_detection.csv', 'a') as f:
            writer = csv.writer(f)
            for j in response['measurements']:
                writer.writerows([[j]])
    else:
        print("Error!")
        print(response)
        print(df_provison_full.loc[i])


Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1

Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1

In [28]:
df_not_global_ip = df[(df.local_resolver == 'True') & (df.rslv_ip_is_global == False) & (df.centralized_rslv == False)].groupby(['prb_id', 'address_family']).agg({'resolver_address':lambda x: set(x)}).reset_index()


list_of_probes4 = df_not_global_ip[df_not_global_ip.address_family == '4'].prb_id.tolist()

list_of_probes6 = df_not_global_ip[df_not_global_ip.address_family == '6'].prb_id.tolist()

source = AtlasSource(
    type = "probes",
    value = ','.join(str(id) for id in list_of_probes4),
    requested = len(list_of_probes4)
)
msm_list = []

#msm_list.append(Traceroute(af=6, target='google.com', description="ICMP Traceroute IPv6 to google.com.", resolve_on_probe=True, protocol="ICMP", packets=1, size=0, first_hop=1, max_hops=64, paris=16, port=80))

msm_list.append(Traceroute(af=4, target='google.com', description="ICMP Traceroute IPv4 to google.com.", resolve_on_probe=True, protocol="ICMP", packets=1, size=0, first_hop=1, max_hops=64, paris=16, port=80))


atlas_request = AtlasCreateRequest(
    start_time = datetime.utcnow() + timedelta(minutes=2),
    key = ATLAS_API_KEY,
    measurements = msm_list,
    sources = [source],
    is_oneoff = True)

(is_success, response) = atlas_request.create()


if is_success:
    print("Success!")
    print(len(response['measurements']))
    #If successful write msm_id to csv
    with open('metadata/msm_id_list_traces_to_local_private_resolvers.csv', 'a') as f:
        writer = csv.writer(f)
        for j in response['measurements']:
            writer.writerows([[j]])
else:
    print("Error!")
    print(response)
    
source = AtlasSource(
    type = "probes",
    value = ','.join(str(id) for id in list_of_probes6),
    requested = len(list_of_probes6)
)
msm_list = []

msm_list.append(Traceroute(af=6, target='google.com', description="ICMP Traceroute IPv6 to google.com.", resolve_on_probe=True, protocol="ICMP", packets=1, size=0, first_hop=1, max_hops=64, paris=16, port=80))

#msm_list.append(Traceroute(af=4, target='google.com', description="ICMP Traceroute IPv4 to google.com.", resolve_on_probe=True, protocol="ICMP", packets=1, size=0, first_hop=1, max_hops=64, paris=16, port=80))


atlas_request = AtlasCreateRequest(
    start_time = datetime.utcnow() + timedelta(minutes=2),
    key = ATLAS_API_KEY,
    measurements = msm_list,
    sources = [source],
    is_oneoff = True)

(is_success, response) = atlas_request.create()


if is_success:
    print("Success!")
    print(len(response['measurements']))
    #If successful write msm_id to csv
    with open('metadata/msm_id_list_traces_to_local_private_resolvers.csv', 'a') as f:
        writer = csv.writer(f)
        for j in response['measurements']:
            writer.writerows([[j]])
else:
    print("Error!")
    print(response)

Success!
1
Success!
1


In [11]:

#
#   IPv4 capable home probes
#

#Read probes from a file
with open("probe_v4_id_list.csv", "r") as f:
    reader = csv.reader(f)
    tmp_list = list(reader)
    id_list_v4 = [item for sublist in tmp_list for item in sublist]



print('Number of IPv4 probes:')
print(len(id_list_v4))


#
#   IPv6 capable home probes
#

#Read probes from a file
with open("probe_v6_id_list.csv", "r") as f:
    reader = csv.reader(f)
    tmp_list = list(reader)
    id_list_v6 = [item for sublist in tmp_list for item in sublist]

print('Number of IPv6 probes:')
print(len(id_list_v6))





#
#   Get IPv4 resolver addresses
#
with open("resolver_list_v4.txt", 'r') as f:
    reader = csv.reader(f)
    tmp_list = list(reader)
    resolver_list_v4 = [sublist[0] for sublist in tmp_list]

#
#   Get IPv6 resolver addresses
#
with open("resolver_list_v6.txt", 'r') as f:
    reader = csv.reader(f)
    tmp_list = list(reader)
    resolver_list_v6 = [sublist[0] for sublist in tmp_list]



Number of IPv4 probes:
2492
Number of IPv6 probes:
1090


In [13]:
len(set(id_list_v6).union(set(id_list_v4)))

2502

In [30]:
for resolver in resolver_list_v4:
    print(resolver)
    
for resolver in resolver_list_v6:
    print(resolver)

1.1.1.1
9.9.9.9
185.228.168.168
156.154.70.1
64.6.64.6
77.88.8.8
8.8.8.8
208.67.222.123
185.121.177.177
45.90.28.0
2606:4700:4700::1111
2620:fe::9
2a0d:2a00:1::1
2610:a1:1018::1
2620:74:1b::1:1
2a02:6b8::feed:0ff
2001:4860:4860::8888
2620:0:ccc::2
2a05:dfc7:5::5353
2a07:a8c0::


In [31]:

#
#   Create source based on probes for IPv4 measurements
#
source_v4 = AtlasSource(
    type = "probes",
    value = ','.join(str(x) for x in id_list_v4),
    requested = len(id_list_v4)
)
#
#   Create source based on probes for IPv6 measurements
#
source_v6 = AtlasSource(
    type = "probes",
    value = ','.join(str(x) for x in id_list_v6),
    requested = len(id_list_v6)
)



for resolver in resolver_list_v4:
    msm_list = []

    msm_list.append(Traceroute(af=4, target=resolver, description="ICMP Traceroute IPv4 to " + resolver + ".", resolve_on_probe=True, protocol="ICMP", packets=1, size=0, first_hop=1, max_hops=64, paris=16))
    
       
    atlas_request = AtlasCreateRequest(
        start_time = datetime.utcnow() + timedelta(minutes=10),
        key = ATLAS_API_KEY,
        measurements = msm_list,
        sources = [source_v4],
        is_oneoff = True)
    
    (is_success, response) = atlas_request.create()
        
        
    if is_success:
        print("Success!")
        print(len(response['measurements']))
        #If successful write msm_id to csv
        with open('metadata/msm_id_list_traces_to_resolvers.csv', 'a') as f:
            writer = csv.writer(f)
            for j in response['measurements']:
                writer.writerows([[j]])
    else:
        print("Error!")
        print(response)
        


for resolver in resolver_list_v6:
    msm_list = []

    msm_list.append(Traceroute(af=6, target=resolver, description="ICMP Traceroute IPv6 to " + resolver + ".", resolve_on_probe=True, protocol="ICMP", packets=1, size=0, first_hop=1, max_hops=64, paris=16))

       
    atlas_request = AtlasCreateRequest(
        start_time = datetime.utcnow() + timedelta(minutes=10),
        key = ATLAS_API_KEY,
        measurements = msm_list,
        sources = [source_v6],
        is_oneoff = True)
    
    (is_success, response) = atlas_request.create()
        
        
    if is_success:
        print("Success!")
        print(len(response['measurements']))
        #If successful write msm_id to csv
        with open('metadata/msm_id_list_traces_to_resolvers.csv', 'a') as f:
            writer = csv.writer(f)
            for j in response['measurements']:
                writer.writerows([[j]])
    else:
        print("Error!")
        print(response)
        



Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1
Success!
1


In [41]:
from ripe.atlas.cousteau import AtlasResultsRequest

from ripe.atlas.sagan import TracerouteResult

In [32]:
tracroute_files = ['metadata/msm_id_list_traces_to_local_resolvers_with_af_detection.csv',
                   'metadata/msm_id_list_traces_to_resolvers.csv']


In [33]:
probes_v4 = pd.read_csv('metadata/probe_v4_id_list.csv', header=None)[0].unique()
probes_v6 = pd.read_csv('metadata/probe_v6_id_list.csv', header=None)[0].unique()

In [34]:
probes_v4

array([10001, 10006, 10015, ..., 55833, 55841, 55844])

In [38]:
msm_ids_trc_to_public_resolvers = pd.read_csv(tracroute_files[1], header=None)[0].unique()

msm_ids_trc_to_local_public_resolvers = pd.read_csv(tracroute_files[0], header=None)[0].unique()

In [45]:
with open('data/traceroute-results-local.csv', 'w+') as f:

    writer = csv.writer(f, delimiter='|')

    writer.writerow(['msm_id', 'timestamp', 'method', 'probe_id', 'addr_fam',
                     'origin', 'src', 'dst', 'dst_name',
                     'status',
                     'ip_path_length',
                     'ttl', 'endpoint', 'rtt'])

    for msm in msm_ids_trc_to_local_public_resolvers:

        kwargs = {
            "msm_id": msm#,
            #"probe_ids": [probes]
        }

        is_success, results = AtlasResultsRequest(**kwargs).create()
         
        
        for result in results:
            
            #print(result)
            res = TracerouteResult(result)
                

            msm_id = res.measurement_id            
            timestamp = res.created_timestamp
            method = res.protocol.lower() + '-' + res.type
            probe_id = res.probe_id
            
            addr_fam = res.af
            origin = res.origin
            src = res.source_address
            dst = res.destination_address
            dst_name = res.destination_name
            ip_path_length = res.total_hops

            for hop in res.hops:
                
                status = 'OK'
                endpoint = '*'
                rtt = -1
                ttl = hop.index
                
                if not hop.error_message:
                    try:
                        pkt = hop.packets[0]
                        
                        if pkt.error_message:
                            # something went wrong, record error and go to next measurement
                            status = pkt.error_message
                            
                            writer.writerow([msm_id, timestamp, method, probe_id, addr_fam,
                                             origin, src, dst, dst_name,
                                             status,
                                             ip_path_length,
                                             ttl, endpoint, rtt]
                                           )
                            break

#                             print "------------"
#                             print res.raw_data
#                             print pkt.error_message
#                             print pkt.raw_data
#                             print "------------"
                        
                        if pkt.origin:
                            endpoint = hop.packets[0].origin

                        rtt = pkt.rtt
                    except:
                        status = 'no packets to parse'

                    writer.writerow([msm_id, timestamp, method, probe_id, addr_fam,
                                     origin, src, dst, dst_name,
                                     status,
                                     ip_path_length,
                                     ttl, endpoint, rtt]
                                   )

                else:
                    # something went wrong, record error and go to next measurement
                    status = str(hop.raw_data['error'])
                    
                    writer.writerow([msm_id, timestamp, method, probe_id, addr_fam,
                                     origin, src, dst, dst_name,
                                     status,
                                     ip_path_length,
                                     ttl, endpoint, rtt]
                                   )

                    break
                    
#                     print res.raw_data
#                     print res.last_hop_errors
#                     print hop.raw_data['error']


#             break




In [46]:
with open('data/traceroute-results-public.csv', 'w+') as f:

    writer = csv.writer(f, delimiter='|')

    writer.writerow(['msm_id', 'timestamp', 'method', 'probe_id', 'addr_fam',
                     'origin', 'src', 'dst', 'dst_name',
                     'status',
                     'ip_path_length',
                     'ttl', 'endpoint', 'rtt'])

    for msm in msm_ids_trc_to_public_resolvers:

        kwargs = {
            "msm_id": msm#,
            #"probe_ids": [probes]
        }

        is_success, results = AtlasResultsRequest(**kwargs).create()
         
        
        for result in results:
            
            res = TracerouteResult(result)

            msm_id = res.measurement_id            
            timestamp = res.created_timestamp
            method = res.protocol.lower() + '-' + res.type
            probe_id = res.probe_id
            
            addr_fam = res.af
            origin = res.origin
            src = res.source_address
            dst = res.destination_address
            dst_name = res.destination_name
            ip_path_length = res.total_hops

            for hop in res.hops:
                
                status = 'OK'
                endpoint = '*'
                rtt = -1
                ttl = hop.index
                
                if not hop.error_message:
                    try:
                        pkt = hop.packets[0]
                        
                        if pkt.error_message:
                            # something went wrong, record error and go to next measurement
                            status = pkt.error_message
                            
                            writer.writerow([msm_id, timestamp, method, probe_id, addr_fam,
                                             origin, src, dst, dst_name,
                                             status,
                                             ip_path_length,
                                             ttl, endpoint, rtt]
                                           )
                            break

#                             print "------------"
#                             print res.raw_data
#                             print pkt.error_message
#                             print pkt.raw_data
#                             print "------------"
                        
                        if pkt.origin:
                            endpoint = hop.packets[0].origin

                        rtt = pkt.rtt
                    except:
                        status = 'no packets to parse'

                    writer.writerow([msm_id, timestamp, method, probe_id, addr_fam,
                                     origin, src, dst, dst_name,
                                     status,
                                     ip_path_length,
                                     ttl, endpoint, rtt]
                                   )

                else:
                    # something went wrong, record error and go to next measurement
                    status = str(hop.raw_data['error'])
                    
                    writer.writerow([msm_id, timestamp, method, probe_id, addr_fam,
                                     origin, src, dst, dst_name,
                                     status,
                                     ip_path_length,
                                     ttl, endpoint, rtt]
                                   )

                    break
                    
#                     print res.raw_data
#                     print res.last_hop_errors
#                     print hop.raw_data['error']


#             break



In [8]:
pd.read_csv('data/traceroute-results-public.csv', sep='|')

,msm_id,timestamp,method,probe_id,addr_fam,origin,src,dst,dst_name,status,ip_path_length,ttl,endpoint,rtt
0,27951715,1604415791,icmp-traceroute,10001,4,82.95.114.207,192.168.178.30,1.1.1.1,1.1.1.1,OK,6,1,192.168.178.1,0.892
1,27951715,1604415791,icmp-traceroute,10001,4,82.95.114.207,192.168.178.30,1.1.1.1,1.1.1.1,OK,6,2,194.109.5.177,19.519
2,27951715,1604415791,icmp-traceroute,10001,4,82.95.114.207,192.168.178.30,1.1.1.1,1.1.1.1,OK,6,3,194.109.7.69,5.317
3,27951715,1604415791,icmp-traceroute,10001,4,82.95.114.207,192.168.178.30,1.1.1.1,1.1.1.1,OK,6,4,194.109.5.5,5.076
4,27951715,1604415791,icmp-traceroute,10001,4,82.95.114.207,192.168.178.30,1.1.1.1,1.1.1.1,OK,6,5,80.249.211.140,11.366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311283,27951734,1604415841,icmp-traceroute,55713,6,2a00:79c0:605:9501:1:eff:fe43:e3d9,2a00:79c0:605:9501:1:eff:fe43:e3d9,2a07:a8c0::,2a07:a8c0::,OK,11,7,2001:ac8:10:10::1a3,28.189
311284,27951734,1604415841,icmp-traceroute,55713,6,2a00:79c0:605:9501:1:eff:fe43:e3d9,2a00:79c0:605:9501:1:eff:fe43:e3d9,2a07:a8c0::,2a07:a8c0::,OK,11,8,2001:ac8:10:10::303,27.819
311285,27951734,1604415841,icmp-traceroute,55713,6,2a00:79c0:605:9501:1:eff:fe43:e3d9,2a00:79c0:605:9501:1:eff:fe43:e3d9,2a07:a8c0::,2a07:a8c0::,OK,11,9,2a0d:5642:115::2,28.373
311286,27951734,1604415841,icmp-traceroute,55713,6,2a00:79c0:605:9501:1:eff:fe43:e3d9,2a00:79c0:605:9501:1:eff:fe43:e3d9,2a07:a8c0::,2a07:a8c0::,OK,11,10,*,NaN


In [10]:
conn = sqlite3.connect('data/ripe-traceroute.db')
pd.read_csv('data/traceroute-results-public.csv', sep='|').to_sql('traceroute', con=conn, index=False, if_exists='replace')
conn.close()